In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer

In [ ]:
max_words = 20
batch_size = 32
epochs = 5

In [ ]:
import pandas
df_train = pandas.read_csv("data/train.csv")
df_test = pandas.read_csv("data/test.csv")
df_val = pandas.read_csv("data/val.csv")

In [ ]:
df_train.head()

In [ ]:
print("Preparing the Tokenizer...")
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train["text"])

In [ ]:
print('Vectorizing sequence data...')
x_train = tokenizer.texts_to_matrix(df_train["text"], mode='binary')
x_test = tokenizer.texts_to_matrix(df_test["text"], mode='binary')
x_val = tokenizer.texts_to_matrix(df_val["text"], mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
x_train

In [ ]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)
print('y_train shape:', y_train.shape)
print('y_val shape:', y_val.shape)

In [ ]:
y_val

In [ ]:
print('Building model sequentially 1...')
model = Sequential()
model.add(Dense(16, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
print('Building model sequentially 2...')
model = Sequential([
          Dense(16, input_shape=(max_words,)),
          Activation('relu'),
          Dense(num_classes),
          Activation('softmax')
        ])

In [ ]:
model.layers

In [ ]:
print(model.to_yaml())

In [ ]:
print('Building model functionally...')
a = Input(shape=(max_words,))
b = Dense(16)(a)
b = Activation('relu')(b)
b = Dense(num_classes)(b)
b = Activation('softmax')(b)
model = Model(inputs=a, outputs=b)

In [ ]:
from keras.models import model_from_yaml

yaml_string = model.to_yaml()
model = model_from_yaml(yaml_string)

In [ ]:
! pip install pydot-ng

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
from keras.objectives import categorical_crossentropy
from keras import backend as K

epsilon = 1.0e-9
def custom_objective(y_true, y_pred):
    '''Yet another crossentropy'''
    y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    cce = categorical_crossentropy(y_pred, y_true)
    return cce

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.compile(loss=custom_objective,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import TensorBoard  
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
from keras.callbacks import EarlyStopping  
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

In [ ]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

In [ ]:
prediction = df_test.copy()
prediction["class"] = pandas.Series(results[:,1])

In [ ]:
prediction[["id", "class"]].to_csv("data/submission.csv", index=False)